In [1]:
import os
import re
import numpy as np
import pandas as pd
from PIL import Image
from segment_anything import sam_model_registry, SamPredictor

In [2]:

# Define paths for loading and saving images -> from optimized version
input_dir = r"C:\Users\dave-\OneDrive - ZHAW\HS24\MoIm\MolecularIMaging\Images\Originals"
output_dir = r"C:\Users\dave-\OneDrive - ZHAW\HS24\MoIm\MolecularIMaging\Images\Output"
phase_mask_dir = os.path.join(output_dir, "phase_masks")
os.makedirs(output_dir, exist_ok=True)

In [3]:
# Load SAM Model
model_type = "vit_h"  # Model type can be "vit_h", "vit_l", or "vit_b"
sam = sam_model_registry[model_type](checkpoint="./sam_vit_h_4b8939.pth")
predictor = SamPredictor(sam)

c:\Users\dave-\anaconda3\envs\MoIm\Lib\site-packages\segment_anything\build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


In [4]:
def get_identifier(filename: str) -> str:
    """
    Extract a unique identifier from a filename based on a pattern.
    
    Args:
        filename (str): The filename of the image.
    
    Returns:
        str: A unique identifier combining the first two parts and last three digits of the filename.
        Returns None if the filename doesn't match the expected pattern.
    """
    match = re.match(r"^([A-H]\d+_\d+).*_(\d{3})\.tif$", filename)
    return f"{match.group(1)}_{match.group(2)}" if match else None

In [5]:
def process_images(input_dir: str, output_dir: str, 
                   save_masks: bool = False, save_masked_images: bool = False) -> pd.DataFrame:
    """
    Process pairs of 'Phase Contrast' and 'GFP' images to calculate GFP intensity within 
    the segmented zebrafish region and save results in a DataFrame.
    
    The function:
    1. Matches pairs of 'Phase Contrast' and 'GFP' images based on unique identifiers.
    2. Generates a binary mask of the zebrafish in the 'Phase Contrast' image using SAM.
    3. Calculates mean and total GFP intensity within the masked region of the corresponding GFP image.
    4. Saves the results in a DataFrame.
    5. Optionally saves the generated mask and masked GFP images.
    
    Args:
        input_dir (str): Directory containing the input TIF images.
        output_dir (str): Directory to save output files (masks and results).
        save_masks (bool): If True, saves the generated masks as PNG files.
        save_masked_images (bool): If True, saves the masked GFP images.

    Returns:
        pd.DataFrame: A DataFrame with GFP intensity data for each image pair.
    """
    # Set phase_mask_dir only if save_masks is True
    phase_mask_dir = os.path.join(output_dir, "phase_masks")
    if save_masks:
        os.makedirs(phase_mask_dir, exist_ok=True)

    # Collect Phase Contrast and GFP files based on unique identifier
    phase_contrast_files = {}
    gfp_files = {}
    
    # Iterate through files to categorize Phase Contrast and GFP images by unique identifier
    for file_name in os.listdir(input_dir):
        if file_name.endswith(".tif"):
            identifier = get_identifier(file_name)
            if identifier:
                if "Phase Contrast" in file_name:
                    phase_contrast_files[identifier] = file_name
                elif "GFP" in file_name:
                    gfp_files[identifier] = file_name

    # List to store GFP intensity results for each identifier
    gfp_intensity_results = []

    # Process each pair of Phase Contrast and GFP files
    for identifier, phase_file in phase_contrast_files.items():
        if identifier in gfp_files:
            # Load Phase Contrast and GFP images
            phase_path = os.path.join(input_dir, phase_file)
            gfp_path = os.path.join(input_dir, gfp_files[identifier])
            
            # Generate mask from the Phase Contrast image
            phase_image = Image.open(phase_path)
            phase_np = np.array(phase_image)
            
            # Convert from uint16 to uint8 by normalizing the pixel values
            phase_np = (phase_np / phase_np.max() * 255).astype(np.uint8)
            
            # Convert grayscale to RGB by stacking along the third dimension for SAM compatibility
            phase_rgb = np.stack([phase_np] * 3, axis=-1)
            
            # Set the image in SAM model for mask generation
            predictor.set_image(phase_rgb)

            # Define a point on the zebrafish (midpoint of the image)
            input_point = np.array([[phase_rgb.shape[1] // 2, phase_rgb.shape[0] // 2]])  # Midpoint of the image
            input_label = np.array([1])  # Label '1' for foreground

            # Generate mask based on the input point
            masks, scores, _ = predictor.predict(
                point_coords=input_point,
                point_labels=input_label,
                multimask_output=False
            )
            mask = masks[0]  # Retrieve the primary mask
            
            # Optionally save the generated mask as a PNG for visual confirmation
            if save_masks:
                mask_output_path = os.path.join(phase_mask_dir, f"mask_{os.path.splitext(phase_file)[0]}.png")
                Image.fromarray((mask * 255).astype(np.uint8)).save(mask_output_path)

            # Load GFP image and calculate GFP intensity within the mask
            gfp_image = Image.open(gfp_path)
            gfp_np = np.array(gfp_image)

            # Apply the mask to isolate GFP values within the fish
            gfp_masked = gfp_np * mask  # Apply the mask to the GFP image
            gfp_values_within_fish = gfp_np[mask > 0]

            # Calculate mean and total GFP intensity within the fish
            mean_gfp_intensity = gfp_values_within_fish.mean()
            total_gfp_intensity = gfp_values_within_fish.sum()

            # Optionally save the masked GFP image
            if save_masked_images:
                output_path = os.path.join(output_dir, f"masked_{gfp_files[identifier]}.png")
                Image.fromarray(gfp_masked).save(output_path)

            # Append the results to the list
            gfp_intensity_results.append({
                "Identifier": identifier,
                "Mean_GFP_Intensity": mean_gfp_intensity,
                "Total_GFP_Intensity": total_gfp_intensity
            })

    # Convert the results to a DataFrame
    results_df = pd.DataFrame(gfp_intensity_results)
    
    # Save DataFrame to CSV
    csv_output_path = os.path.join(output_dir, "gfp_intensity_results.csv")
    results_df.to_csv(csv_output_path, index=False)
    
    print(f"Results saved to {csv_output_path}")
    
    # Return the DataFrame with results
    return results_df

In [6]:
# Run the function on the directory with options for saving images and masked images
gfp_intensity_results_df = process_images(input_dir, output_dir, save_masks=True, save_masked_images=True)

KeyboardInterrupt: 